# Определение стоимости автомобилей

## Описание исследования

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Цель исследования

- Создать регрессионную модель машинного обучения, которая будет способна прогнозировать рыночную стоимость автомобиля по данным

## Ход исследования

1. Получение данных
2. Предобработка данных
3. Исследовательский анализ
4. Подготовка данных
5. Обучение моделей
6. Выводы

<a id='section_id'></a>
## Содержание 

[Шаг 1. Загрузка данных](#section_id1)

[Шаг 2. Предобработка данных](#section_id2)

[Шаг 3. Подготовка данных](#section_id3)

[Шаг 4. Обучение моделей](#section_id4)

[Шаг 5. Анализ результатов](#section_id5)

[Шаг 6. Выбор лучшей модели](#section_id6)

[Общий вывод](#section_id7)

In [ ]:
# установка библиотек
!pip install phik -q
!pip install category_encoders -q

In [ ]:
# импорт библиотек

# работа с данными
import pandas as pd
import numpy as np

# графика
import matplotlib.pyplot as plt
import seaborn as sns

# инструменты анализа
from phik.report import plot_correlation_matrix
from phik import phik_matrix

# подготовка данных
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder

# пайплайны
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# модели машинного обучения
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor

# инструменты управления ресурсами
import joblib

# инструменты поиска
from sklearn.model_selection import GridSearchCV

# 
from sklearn.metrics import mean_squared_error

In [ ]:
# константы
TEST_SIZE = 0.25 
RANDOM_STATE = 42

<a id='section_id1'></a>
## Шаг 1. Загрузка данных
[к содержанию](#section_id)

In [ ]:
# загрузка данных
df = pd.read_csv('/datasets/autos.csv', parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])
df.head()

<a id='section_id2'></a>
## Шаг 2. Предобработка данных
[к содержанию](#section_id)

### Корректировка названий столбцов

In [ ]:
# функция для snake_case
def make_snake(header):
    '''Функция принимает на вход заголовок и возвращает snake_case'''
    result = ''
    
    # поставить перед всеми словами нижнее подчеркивание
    for letter in header:
        if letter.isupper():
            result += '_' + letter.lower()
        else:
            result += letter
            
    # удалить нижнее подчеркивание в начале слова
    if result[0] == '_':
        result = result[1:]
    return result

In [ ]:
# корректировка названий столбцов
df.columns = [make_snake(header) for header in df.columns]

### Обзор данных

In [ ]:
# функция для обзора данных
def preview(dataset):
    '''Функция принимает на вход набор данных и выводит основную информацию о нем.'''
    display(dataset.head())
    dataset.info()
    display(dataset.describe(include='all', datetime_is_numeric=True).T)

In [ ]:
# обзор данных
preview(df)

### Удаление неинформативных признаков

К удалению предлагаю:

- registration_month - месяц регистрации автомобиля
- date_crawled — дата скачивания анкеты из базы
- date_created — дата создания анкеты
- number_of_pictures — количество фотографий автомобиля
- postal_code — почтовый индекс владельца анкеты (пользователя)
- last_seen — дата последней активности пользователя

Даже если признаки коррелируют с целевым, не логично прогнозировать по ним стоимость автомобиля.

In [ ]:
# удаление неинформативных признаков
df = df[['registration_year',
         'price',
         'vehicle_type',
         'gearbox',
         'power',
         'model',
         'kilometer',
         'fuel_type',
         'brand',
         'repaired']]

### Заполнение пропусков

In [ ]:
# определение столбцов с пропусками
nulls = pd.DataFrame({'nulls' : df.isna().sum(), 'percent' : round(df.isna().mean()*100)})
nulls

In [ ]:
# просмотр строк с пропусками
df[df.isna().any(axis=1)].sample(10)

Пропуски в model и vehicle_type можно заменить значением 'other'.

Пропуски по категориальным признакам repaired, fuel_type, gearbox заполним значениями 'unknown'

In [ ]:
# заполнение пропусков
df[['vehicle_type', 'fuel_type']] = df[['vehicle_type', 'fuel_type']].fillna('other')
df[['repaired', 'model', 'gearbox']] = df[['repaired', 'model', 'gearbox']].fillna('unknown')

### Исследовательский анализ

- vehicle_type — тип автомобильного кузова - категориальный признак, предпочтительно кодирование OHE
- gearbox — тип коробки передач - категориальный признак, предпочтительно кодирование OHE
- power — мощность (л. с.) - количественный признак, необходимо произвести масштабирование
- model — модель автомобиля - категориальный признак, предпочтительно кодирование OHE
- kilometer — пробег (км) - количественный признак, необходимо произвести масштабирование
- fuel_type — тип топлива - категориальный признак, предпочтительно кодирование OHE
- brand — марка автомобиля - категориальный признак, предпочтительно кодирование OHE
- repaired — была машина в ремонте или нет - категориальный признак, предпочтительно кодирование OHE

In [ ]:
# изучение столбца 'fuel_type'
df['fuel_type'].unique()

In [ ]:
# замена значений в столце 'fuel_type'
df['fuel_type'] = df['fuel_type'].str.replace('gasoline', 'petrol')
df['fuel_type'] = df['fuel_type'].str.replace('cng', 'lpg')

### Исследование аномалий

In [ ]:
# списки названий столбцов
num_columns = ['power',
               'kilometer',
               'price',
               'registration_year']
cat_columns = ['vehicle_type',
               'gearbox',
               'model',
               'fuel_type',
               'brand',
               'repaired']

In [ ]:
# гистограмма признака power
sns.histplot(data=df, x='power', bins=10, binwidth=20)
plt.title('Мощность автомобиля')
plt.show()

- В столбце **power** присутствуют аномальные значения. Адекватные значения мощности представленных в датасете автомобилей от 30 до 500 лошаиных сил. Также учтём, что машина может продаваться вообще без двигателя. Оставим в выборке автомобили с мощностью 0 л.с.

In [ ]:
# удаление аномалий
df = df[(df['power'] == 0) |(df['power'] > 30) & (df['power'] < 500)]

In [ ]:
# гистограмма признака power
sns.histplot(data=df, x='power', bins=10, binwidth=20)
plt.title('Мощность автомобиля')
plt.show()

In [ ]:
# диаграмма размаха признака kilometer
sns.boxplot(data=df, y='kilometer')
plt.title('Пробег автомобиля')
plt.show()

- Значения в столбце **kilometer** находятся в адекватных пределах, пробег 140000 км для подержанных машин - это неплохой результат

In [ ]:
# графики признака price
fig, ax = plt.subplots(1,2, figsize=(10, 5))
sns.boxplot(data=df, y='price', ax=ax[0])
sns.histplot(data=df, x='price', ax=ax[1])
plt.suptitle('Цена автомобиля')
plt.show()

- Значения в столбце **price** имеют большое количество значений выше Q3+1.5*IQR. Не будем удалять их из рассмотрения, чтобы не потерять данные.

In [ ]:
# диаграмма размаха признака registration_year
sns.boxplot(data=df, y='registration_year')
plt.title('Год регистрации автомобиля')
plt.show()

- Значения в столбце **registration_year** имеют неадекватные значения года регистрации. Заменим 211 значений выше 2024 года и 99 ниже 1950 на 2024 и 1950 соответственно.

In [ ]:
# Заменим значения года, на корректные
df.loc[df['registration_year'] > 2024, 'registration_year'] = 2024
df.loc[df['registration_year'] < 1950, 'registration_year'] = 1950

In [ ]:
# диаграмма размаха признака registration_year
sns.boxplot(data=df, y='registration_year')
plt.title('Год регистрации автомобиля после изменений')
plt.show()

In [ ]:
# столбчатая диаграмма признака vehicle_type
sns.barplot(x=df['vehicle_type'].unique(), y=df['vehicle_type'].value_counts())
plt.title('Тип кузова')
plt.show()

In [ ]:
# столбчатая диаграмма признака gearbox
sns.barplot(x=df['gearbox'].unique(), y=df['gearbox'].value_counts())
plt.title('Тип коробки передач')
plt.show()

In [ ]:
# столбчатая диаграмма признака model (5 наиболее часто встречаемых моделей)
sns.barplot(x=df['model'].unique()[:5], y=df['model'].value_counts()[:5])
plt.title('5 наиболее часто встречаемых моделей авто')
plt.show()

In [ ]:
# столбчатая диаграмма признака fuel_type
sns.barplot(x=df['fuel_type'].unique(), y=df['fuel_type'].value_counts())
plt.title('Количество авто по типу топлива')
plt.show()

In [ ]:
# столбчатая диаграмма признака brand (5 наиболее часто встречаемых брендов авто)
sns.barplot(x=df['brand'].unique()[:5], y=df['brand'].value_counts()[:5])
plt.title('5 наиболее часто встречаемых брендов авто')
plt.show()

In [ ]:
# столбчатая диаграмма признака repaired
sns.barplot(x=df['repaired'].unique(), y=df['repaired'].value_counts())
plt.title('Количество авто по состоянию')
plt.show()

### Корреляционный анализ

In [ ]:
# анализ корреляционных зависимостей между признаками
phik_overview = df.phik_matrix(verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    figsize=(12, 12)
)

По шкале Чеддока в средней корреляции с целевым находятся признаки model, power, registration_year слабой связью с целевым обладают repaired, brand, kilometer, gearbox, очень слабая связь у fuel_type

Присутствует мультиколлинеарность между признаками model и brand, которую необходимо устранить.
Удалим признак model.

In [ ]:
# удаление столбца brand
df = df[['price',
         'registration_year',
         'vehicle_type',
         'gearbox',
         'power',
         'brand',
         'kilometer',
         'fuel_type',
         'repaired']]

<a id='section_id3'></a>
## Шаг 3. Подготовка данных
[к содержанию](#section_id)

In [ ]:
# создание тренировочной и тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['price'], axis=1),
    df['price'],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE)

X_train.shape, X_test.shape

In [ ]:
# список num_columns
num_columns = ['power',
               'kilometer',
               'registration_year']

# список te_columns
te_columns = ['vehicle_type',
               'gearbox',
               'brand',
               'fuel_type',
               'repaired']

In [ ]:
# пайплайн для кодирования ohe_columns
te_pipe = Pipeline(
    [('te', TargetEncoder())]
    )

In [ ]:
# общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('te', te_pipe, te_columns),
     ('num', MinMaxScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
# итоговый пайплайн
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LinearRegression())
])

In [ ]:
# задание параметров для пайплайна
param_grid = [
    # словарь для модели decisionTreeRegressor()
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_features': range(3, 5),
        'models__max_depth': range(5, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough'] 
    },
    # словарь для модели LightGBM()
    {
        'models': [LGBMRegressor(n_jobs=-1)],
        'models__max_depth': [4, 5],
        'models__learning_rate': [1.0, 10.0],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]

<a id='section_id4'></a>
## Шаг 4. Обучение моделей
[к содержанию](#section_id)

In [ ]:
# инициализация подбора параметров
grid_search = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

In [ ]:
# настройка библиотеки для работы с многопоточными вычислениями
joblib.parallel_backend('threading')

In [ ]:
%%time
# запуск подбора параметров
grid_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid_search.best_score_)

<a id='section_id5'></a>
## Шаг 5. Анализ результатов
[к содержанию](#section_id)

In [ ]:
# анализ важности признаков
grid_search.best_estimator_._final_estimator.feature_importances_

In [ ]:
# получение результатов
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.head()

In [ ]:
# выбор лучшей модели DecisionTreeRegressor
best_tree = results[results['param_models'].astype('str').str.contains('Decision')].iloc[0]
best_tree

In [ ]:
# выбор лучшей модели LGBMRegressor
best_lgbm = results[results['param_models'].astype('str').str.contains('LGBM')].iloc[0]
best_lgbm

**Анализ времени обучения (mean_fit_time):**
- DecisionTreeRegressor - 0.82 секунды
- LGBMRegressor - 5.06 секунды

**Анализ времени предсказания (mean_score_time):**
- DecisionTreeRegressor - 52 миллисекунды
- LGBMRegressor - 484 миллисекунды

**Анализ качества моделей (mean_test_score)**
- Метрика лучшей модели DecisionTreeRegressor на тестовой выборке: 2185.94
- Метрика лучшей модели LGBMRegressor на тестовой выборке: 1895.89

<a id='section_id6'></a>
## Шаг 6. Выбор лучшей модели
[к содержанию](#section_id)

С точки зрения заказчика лучшими характеристиками обладает модель:

    mean_fit_time                                                           0.823442
    mean_score_time                                                         0.052086
    param_models                              DecisionTreeRegressor(random_state=42)
    mean_test_score                                                     -2185.941665
    rank_test_score                                                                7

Модель DecisionTreeRegressor побеждает в сравнении с LGBMRegressor в плане скорости обучения и предсказания, и незначительно проигрывает в качестве.

In [ ]:
# трансформация данных
grid_search.best_estimator_.named_steps['preprocessor'].fit(X_train, y_train)
X_train_transformed = grid_search.best_estimator_.named_steps['preprocessor'].transform(X_train)
X_test_transformed = grid_search.best_estimator_.named_steps['preprocessor'].transform(X_test)

In [ ]:
# запись лучшей модели в переменную
best_tree_model = results.loc[28, 'param_models']

In [ ]:
# обучение лучшей модели
best_tree_model.fit(X_train_transformed, y_train)

In [ ]:
# расчет метрики на тестовой выборке для LGBMRegressor
y_test_pred_lgbm = best_tree_model.predict(X_test_transformed)
print('Метрика лучшей модели LGBMRegressor на тестовой выборке:', 
      mean_squared_error(y_test, y_test_pred_lgbm, squared=False))

<a id='section_id7'></a>
## Общий вывод
[к содержанию](#section_id)

В ходе выполнения проекта была решена задача регрессии по прогнозированию стоимости подержанных автомобилей на основе данных.

Была выполнена предобработка данных:
- удалены неинформативные признаки: registration_month, date_crawled, date_created, number_of_pictures, postal_code, last_seen
- заполнены пропуски категориальных признаков категориями 'other' и 'unknown'
- в ходе корреляционного анализа обнаружена мультиколинеарность между признаками 'brand' и 'model'

С помощью пайплайна были подобраны лучшие параметры для моделей трёх классов: DecisionTreeRegressor, LGBMRegressor. Для оценки качества использовалась метрика RMSE.

Выбор о рекомендации заказчику модели был сделан в пользу скорости обучения и предсказания модели. На тестовых данных лучшая модель DecisionTreeRegressor показала допустимое значение метрики RMSE в 2136.52.